# Imitation learning training script!!!!

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim

## Check if CUDA is available

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


## Data loader

In [4]:
def load_training_data(file_path):
    return 0

## Design Model

In [ ]:
class JackDrivingModel(nn.Module):

    def __init__(self, img_channels = 3, img_h = 600, img_w = 800, act_dim = 2):
        super().__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(img_channels, 24, kernel_size=5, stride=2), nn.ReLU(),
            nn.Conv2d(24, 36, kernel_size=5, stride=2), nn.ReLU(),
            nn.Conv2d(36, 48, kernel_size=5, stride=2), nn.ReLU(),
            nn.Conv2d(48, 64, kernel_size=3, stride=1), nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1), nn.ReLU()
        )

    with torch.no_grad():
            dummy = torch.zeros(1, img_channels, img_h, img_w)
            conv_out = self.conv(dummy)
            conv_feat_dim = conv_out.view(1, -1).shape[1]

    self.fc = nn.Sequential(
        nn.Linear(conv_feat_dim, 100), nn.ReLU(),
        nn.Linear(100, 50), nn.ReLU(),
        nn.Linear(50, action_dim)  # [vx, wz]
    )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

        

IndentationError: unindent does not match any outer indentation level (<string>, line 19)